# Report handler examples

Install leakpro as ``` pip install -e /path/to/leakpro ```

### Synthetic examples

In [1]:
import os
import sys

import pandas as pd

sys.path.append("../..")

from leakpro.synthetic_data_attacks.anomalies import return_anomalies
from leakpro.synthetic_data_attacks.inference_utils import inference_risk_evaluation
from leakpro.synthetic_data_attacks.linkability_utils import linkability_risk_evaluation
from leakpro.synthetic_data_attacks.singling_out_utils import singling_out_risk_evaluation

#Get ori and syn
n_samples = 100
DATA_PATH = "../synthetic_data/datasets/"
ori = pd.read_csv(os.path.join(DATA_PATH, "adults_ori.csv"), nrows=n_samples)
syn = pd.read_csv(os.path.join(DATA_PATH, "adults_syn.csv"), nrows=n_samples)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
syn_anom = return_anomalies(df=syn, n_estimators=1000, n_jobs=-1, verbose=True)
print("Syn anom shape",syn_anom.shape)

[Parallel(n_jobs=64)]: Using backend ThreadingBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done   2 out of  64 | elapsed:    0.9s remaining:   28.2s
[Parallel(n_jobs=64)]: Done  64 out of  64 | elapsed:    4.3s finished


Unique predictions (array([-1,  1]), array([ 3, 97]))
Syn anom shape (3, 14)


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.1s


In [3]:
# Create a singling-out result
sin_out_res = singling_out_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    n_attacks = syn_anom.shape[0]
)

In [4]:
# Create linkability result
link_res = linkability_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    n_samples = syn_anom.shape[0],
    n_attacks = 100
)

In [5]:
# # Create base-case inference result
inf_res = inference_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    worst_case_flag = False,
    n_attacks = syn_anom.shape[0]
)

# # Create worst-case inference result
inf_res_worst = inference_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    worst_case_flag = True,
    n_attacks = syn_anom.shape[0]
)

### Gradient inversion example

In [6]:
sys.path.append("../gia/inverting_cifar10_1_image/")
from cifar import get_cifar10_loader
from model import ResNet
from torchvision.models.resnet import BasicBlock

from leakpro.attacks.gia_attacks.utils import InvertingConfig
from leakpro.metrics.attack_result import GIAResults
from leakpro.fl_utils.gia_train import train
from leakpro.run import run_inverting

model = ResNet(BasicBlock, [5, 5, 5], num_classes=10, base_width=16 * 10)
client_dataloader, data_mean, data_std = get_cifar10_loader(num_images=1, batch_size=1, num_workers=2)

# Meta train function designed to work with GIA
train_fn = train

# Baseline config
configs = InvertingConfig()
configs.at_iterations = 2000 # Decreased from 8000 to avoid GPU memory crash

name = "my_gia_results"
GIA_result_generator = run_inverting(model, client_dataloader, train_fn, data_mean, data_std, configs, experiment_name=name, save=False)

Files already downloaded and verified


2025-03-25 17:24:39,994 INFO     Inverting gradient initialized.


In [7]:
GIAResults = GIAResults.collect_generator(GIA_result_generator)

2025-03-25 17:24:41,028 INFO     New best loss: 0.0517808273434639 on round: 0
2025-03-25 17:24:41,029 INFO     Iteration 0, loss 0.0517808273434639
/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:70: FutureWarning: Importing `peak_signal_noise_ratio` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `peak_signal_noise_ratio` from `torchmetrics.image` instead.
  _future_warning(
2025-03-25 17:24:41,625 INFO     New best loss: 0.04392379894852638 on round: 1
2025-03-25 17:24:41,721 INFO     New best loss: 0.03822696954011917 on round: 2
2025-03-25 17:24:41,809 INFO     New best loss: 0.03492433950304985 on round: 3
2025-03-25 17:24:41,893 INFO     New best loss: 0.03250207006931305 on round: 4
2025-03-25 17:24:41,977 INFO     New best loss: 0.031010817736387253 on round: 5
2025-03-25 17:24:42,062 INFO     New best loss: 0.029381880536675453 on round: 6
2025-03-25 17:24:42,144 INFO     New best loss: 0.028015263378620148 on round: 7

In [8]:
# GIA_result_generator = run_inverting(model, client_dataloader, train_fn, data_mean, data_std, configs, experiment_name=name, save=False)
# i_list, score_list, GIA_result_list = zip(*GIA_result_generator)


### Membership Inference Attack, CIFAR example

In [9]:
import os
import sys
import yaml

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(project_root)

In [10]:
from utils.cifar_data_preparation import get_cifar_dataloader
from utils.cifar_model_preparation import ResNet18, create_trained_model_and_metadata


# Load the config.yaml file
with open('train_config.yaml', 'r') as file:
    train_config = yaml.safe_load(file)

# Generate the dataset and dataloaders
path = os.path.join(os.getcwd(), train_config["data"]["data_dir"])

train_loader, test_loader = get_cifar_dataloader(path, train_config)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
# Train the model
if not os.path.exists("target"):
    os.makedirs("target")
if train_config["data"]["dataset"] == "cifar10":
    num_classes = 10
elif train_config["data"]["dataset"] == "cifar100":
    num_classes = 100
else:
    raise ValueError("Invalid dataset name")

model = ResNet18(num_classes = num_classes)
train_acc, train_loss, test_acc, test_loss = create_trained_model_and_metadata(model, 
                                                                               train_loader, 
                                                                               test_loader, 
                                                                               train_config)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Training Progress: 100%|██████████| 10/10 [00:46<00:00,  4.65s/it]


##### Run the MIA attack

In [12]:
from cifar_handler import CifarInputHandler

from leakpro import LeakPro

# Read the config file
config_path = "audit.yaml"

# Prepare leakpro object
leakpro = LeakPro(CifarInputHandler, config_path)

# Run the audit 
mia_results = leakpro.run_audit(return_results=True)

2025-03-25 17:28:33,917 INFO     Target model blueprint created from ResNet18 in ./utils/cifar_model_preparation.py.
2025-03-25 17:28:33,964 INFO     Loaded target model metadata from ./target/model_metadata.pkl
2025-03-25 17:28:34,171 INFO     Loaded target model from ./target
2025-03-25 17:28:35,266 INFO     Loaded population dataset from ./data/cifar10.pkl
2025-03-25 17:28:35,267 INFO     Loaded population dataset from ./data/cifar10.pkl
2025-03-25 17:28:35,269 INFO     Image extension initialized.
2025-03-25 17:28:35,324 INFO     MIA attack factory loaded.
2025-03-25 17:28:35,326 INFO     Creating shadow model handler singleton
2025-03-25 17:28:35,451 INFO     Creating distillation model handler singleton
2025-03-25 17:28:35,575 INFO     Configuring the RMIA attack
2025-03-25 17:28:35,580 INFO     User provided value for gamma, it won't be optimized by optuna.
2025-03-25 17:28:35,581 INFO     Added attack: rmia
2025-03-25 17:28:35,582 INFO     Shadow model handler singleton already

In [19]:
# # Import and initialize ReportHandler
from leakpro.reporting.report_handler import ReportHandler

# # Set report_dir to "./leakpro_output/results" to the results to a local results folder
#    or don't use the report_dir argument to let the ReportHandler find an already
#    existing results folder
report_handler = ReportHandler(report_dir="./leakpro_output/results")

# Save Synthetic results using the ReportHandler
report_handler.save_results(attack_name="singling_out", result_data=sin_out_res)
report_handler.save_results(attack_name="linkability_risk", result_data=link_res)
report_handler.save_results(attack_name="inference_risk_base", result_data=inf_res)
report_handler.save_results(attack_name="inference_risk_worst", result_data=inf_res_worst)

# Save GIA results using report handler
report_handler.save_results(attack_name="gia", result_data=GIAResults, config=GIAResults.config)

# Save MIA resuls using report handler
for res in mia_results:
    report_handler.save_results(attack_name=res.attack_name, result_data=res, config=res.configs)

2025-03-25 18:18:29,225 INFO     Initializing report handler...
2025-03-25 18:18:29,228 INFO     report_dir set to: ./leakpro_output/results
2025-03-25 18:18:29,230 INFO     Saving results for singling_out
2025-03-25 18:18:31,490 INFO     Saving results for linkability_risk
2025-03-25 18:18:33,524 INFO     Saving results for inference_risk_base
2025-03-25 18:18:38,156 INFO     Saving results for inference_risk_worst
2025-03-25 18:18:40,477 INFO     Saving results for gia
2025-03-25 18:18:40,506 INFO     Saving results for rmia
2025-03-25 18:18:47,519 INFO     Saving results for lira


<Figure size 3300x1200 with 0 Axes>

<Figure size 3300x1200 with 0 Axes>

<Figure size 3300x2100 with 0 Axes>

<Figure size 3300x1200 with 0 Axes>

In [20]:
# Simply create a report by collecting and compiling latex text
report_handler.create_report()

2025-03-25 18:19:48,614 INFO     PDF compiled


<Figure size 1920x1440 with 0 Axes>

<Figure size 3300x1200 with 0 Axes>

<Figure size 3300x2100 with 0 Axes>

<Figure size 3300x2100 with 0 Axes>

<Figure size 3300x1200 with 0 Axes>

In [21]:
report_handler_STEPWISE = ReportHandler(report_dir="./leakpro_output/results")

#### Have more granular control

# Use the ReportHandler and load all the saved results
report_handler_STEPWISE.load_results()

# Create results and collect corresponding latex texts. Use different methods to create results for a specific type
# report_handler_STEPWISE.create_results_mia()
# report_handler_STEPWISE.create_results_gia()
# report_handler_STEPWISE.create_results_syn()
report_handler_STEPWISE.create_results_all()

# Create the report by compiling the latex text
report_handler_STEPWISE.create_report()

2025-03-25 18:22:41,310 INFO     Initializing report handler...
2025-03-25 18:22:41,313 INFO     report_dir set to: ./leakpro_output/results
2025-03-25 18:23:28,214 INFO     PDF compiled


<Figure size 1920x1440 with 0 Axes>

<Figure size 3300x1200 with 0 Axes>

<Figure size 3300x2100 with 0 Axes>

<Figure size 3300x2100 with 0 Axes>

<Figure size 3300x1200 with 0 Axes>